# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [10]:
import pandas as pd

recipes = pd.read_csv('data/recipes_sample.csv',sep=',',header=0,engine='python')

rands = recipes[['id','minutes']].sample(n=5,replace=False)
print(rands.to_string(index=False,justify='center',formatters=[lambda x: f'|{x:^}|',lambda x: f'{x:^}|']))

   id    minutes
|292055|    30| 
|189583|   135| 
|239834|    70| 
|432319|    40| 
 |54615|    20| 


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [18]:
from bs4 import BeautifulSoup as bs
with open('../лаба 3/data/steps_sample.xml','r') as file:
    content = file.read()
    
soop = bs(content,'xml')    

i = soop.find('id',string='170895').find_parent('recipe')
for j in i.find_all('step'):
    steps.append(j.text)

In [104]:


def show_info(name='',steps=[],minutes=0,author_id=0):
    res = f'"{name}"\n\n'
    for i,v in enumerate(steps,1):
        res+= f'{i}. {v}\n'
    res+=f'----------\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут\n'
    return res
    


check = recipes[recipes['id']==170895] # я тут подправлял полсе завершения работы, так что вывод кривой, надо просто перезапустить все ячейки
print(show_info(name=check.iloc[0]['name'],steps = steps,minutes=check.iloc[0]['minutes'],author_id=check.iloc[0]['contributor_id']))

"leeks and parsnips  sauteed or creamed"

1. 

2. mix butter , flour , 1/3 c
3. 

4. sugar and 1-1/4 t
5. 

6. vanilla
7. 

8. press into greased 9" springform pan
9. 

10. mix cream cheese , 1/4 c
11. 

12. sugar , eggs and 1/2 t
13. 

14. vanilla beating until fluffy
15. 

16. pour over dough
17. 

18. combine apples , 1/3 c
19. 

20. sugar and cinnamon
21. 

22. arrange on top of cream cheese mixture and sprinkle with almonds
23. 

24. bake at 350 for 45-55 minutes , or until tester comes out clean
25. 

----------
Автор: 8377
Среднее время приготовления: 27 минут



In [13]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

<IPython.core.display.Javascript object>

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [40]:
steps = []
i = soop.find('id',string='25082').find_parent('recipe')
for j in i.find('steps'):
    steps.append(j.text)
print(steps)

['\n', 'proof yeast in half a cup of the water , with about a teaspoon of flour- let it sit , covered , until it becomes bubbly& active', '\n', 'combine the rest of the water with the yeast / water in a large bowl , and gradually add about half of the flour', '\n', 'stir the mixture about 100 times in the same direction , until it is well-mixed and strands are beginning to form', '\n', 'stir in the salt and oil', '\n', 'add the remaining flour half a cup at a time , stirring well , until it is too stiff to stir with a spoon', '\n', 'turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board', '\n', 'the dough should be springy and nice to work with', '\n', 'when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly', '\n', 'after it has rested , knead the dough a few more times and place it in the bowl , covered with the damp cloth , a pl

In [46]:
import re
regex = re.compile('.*\d+\s[hour,hours,minute,minutes]{1}.*')

print(list(filter(lambda x: bool(regex.match(x)),steps)))

['stir the mixture about 100 times in the same direction , until it is well-mixed and strands are beginning to form', 'turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board', 'when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly', 'let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours', 'when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc', 'bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [67]:
regex = re.compile('this[\s\w]*, but.*')

arg = recipes.dropna()

matches = arg[arg['description'].str.match(regex)]
print(matches.shape[0])
for i,v in enumerate(matches.sample(n=3,replace=False)['description']):
    print(i,v)

52
0 this is a delicious traditional spanish dessert and is similar to cold custard, but creamier and with a hint of cinnamon.
1 this pie crust retains the flavor of an all butter crust, but is flakier because of the shortening. the lemon juice isn't in the recipe for lemon flavour... trust me you won't taste it, but rather to work with the flour and prevent the pastry from being tough. i also find that the dough gives a bigger yield than the original 9 inch size stated, i roll mine relatively thin and can get a full top and bottom for a 26 cm (12 inch) shallow pie dish. i have included 20+ step by step photographs to help you see *exactly* how this is made so that you now have instructions for making an entire pie crust from ingredients to table. the new instructions may look long, but i have only tried to give practical details to that even if this is your first ever pie crust, you will have all the information you need for success. enjoy!  zwt region: united states.
2 this is sooo g

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [84]:
steps = []
i = soop.find('id',string='72367').find_parent('recipe')
for j in i.find('steps'):
    steps.append(j.text)
    
print(steps)
    
for i,_ in enumerate(steps):
    steps[i] = re.sub('\s+/\s+','/',_)
print(steps)

['\n', 'mix butter , flour , 1 / 3 c', '\n', 'sugar and 1-1 / 4 t', '\n', 'vanilla', '\n', 'press into greased 9" springform pan', '\n', 'mix cream cheese , 1 / 4 c', '\n', 'sugar , eggs and 1 / 2 t', '\n', 'vanilla beating until fluffy', '\n', 'pour over dough', '\n', 'combine apples , 1 / 3 c', '\n', 'sugar and cinnamon', '\n', 'arrange on top of cream cheese mixture and sprinkle with almonds', '\n', 'bake at 350 for 45-55 minutes , or until tester comes out clean', '\n']
['\n', 'mix butter , flour , 1/3 c', '\n', 'sugar and 1-1/4 t', '\n', 'vanilla', '\n', 'press into greased 9" springform pan', '\n', 'mix cream cheese , 1/4 c', '\n', 'sugar , eggs and 1/2 t', '\n', 'vanilla beating until fluffy', '\n', 'pour over dough', '\n', 'combine apples , 1/3 c', '\n', 'sugar and cinnamon', '\n', 'arrange on top of cream cheese mixture and sprinkle with almonds', '\n', 'bake at 350 for 45-55 minutes , or until tester comes out clean', '\n']


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [86]:
from nltk.tokenize import  sent_tokenize, word_tokenize

all_stps = soop.find_all('steps')

uniqs = set()

for i in all_stps:
    for j in word_tokenize(i.text):
        if j.isalpha():
            uniqs.update(j)
            
print(len(uniqs))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Валерий Михайлович/nltk_data'
    - 'c:\\python39\\nltk_data'
    - 'c:\\python39\\share\\nltk_data'
    - 'c:\\python39\\lib\\nltk_data'
    - 'C:\\Users\\Валерий Михайлович\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [99]:

recipes['num_sentences'] = pd.Series([len(sent_tokenize(i)) for i in recipes['description']])
recipes.iloc[recipes['num_sentences'].nlargest(5).index]


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
3405,blueberry liqueur,236281,129615,327115,2007-06-21,NaN,this is an absolutely beautiful shade of purpl...,5.0
26046,strawberry liqueur,236274,129615,327115,2007-06-21,7.0,a beautiful light red shade with a delicious s...,5.0
9116,dandelion wine,165697,87842,26657,2006-04-25,6.0,this is just fun to make.,6.0
11720,garden hot sauce,34175,86400,41525,2002-07-15,7.0,this is a recipe from a chef at commander's ki...,3.0
16223,limoncello italian lemoncello or orangecello,93851,86400,54716,2004-06-20,NaN,got this from a family i stayed with in sorren...,4.0


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [ ]:
def magic(arg):
    res,bottom = '',''
    for i in nltk.pos_tag(word_tokenize(arg)):
        res+=f'{i[1]:^len(i[0]+2)}'
        bottom+=f' {i[0]} '
    return res+'\n'+bottom
magic(recipes[recipes['id']==241106]['name'])